<center><img src="https://media3.giphy.com/media/txYzAPM0C83yo/giphy.gif?cid=ecf05e47an8ed3h35rl48abrd7xpxofhcs3ezslr2rywlts4&rid=giphy.gif"></center>

Tomado de: https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a

* And why use Huggingface Transformers instead of Googles own BERT solution? Because with Transformers it is extremely easy to switch between different models, that being BERT, ALBERT, XLnet, GPT-2 etc. Which means, that you more or less ‘just’ replace one model for another in your code

## Load the required libraries

In [35]:
!pip install -q transformers

In [36]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

simpletransformers==0.60.8


In [37]:
import pandas as pd
df = pd.read_excel("TramiteGarantia.xlsx")
df.head()

,Caso,Estado,Estado Gestión,Propietario del caso,Marca de Ocurrencia,Categoria,Vendedor,Nombre de la cuenta,Documento,Email de Contacto,Teléfono de Contacto,Celular de Contacto,Fecha de apertura,Fecha esperada de cierre,Antigüedad,No. de Pedido/Tirilla,Producto,Marca PLU,Descripcion PLU,Fecha de Compra,Origen del caso,Descripción,Creado por,Medio de respuesta,Última modificación de caso realizada por,Medio de devolucion,Fecha de la última modificación de caso,Solución Express,Fecha de cierre,Detalle solución,Descripcion Solucion,Alias del propietario del caso
0,479725,Pendiente por documentación,Reasignado N2,Lina Marcela Bohorquez Gonzalez,EXITO.COM,Marketplace,Colchones Paraiso 1,Kevis Manuel Pitalua Castro,CC-1070813521,elvivekt@hotmail.com,NaN,3.117452e+09,18/11/2020,7/01/2021,1829,1041533392806,100126160,COLCHONES PARAISO,Colchon Paraíso Bora Doble 140x190x29 Una cara...,21/06/2020,Call center,0000030a5170a9ff/TAMEJIHO: Cliente se comunica...,TATIANA ANDREA MEJÍA HOYOS,Correo,Cristian Omar Montiel Cermeño,NaN,2/02/2021,False,NaN,NaN,NaN,linaboho
1,590248,Pendiente por documentación,Pendiente Respuesta del Cliente,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Bejarano Sandra Patric Beltran,CC-40447782,jsmarin04@hotmail.com,NaN,3.118258e+09,30/01/2021,15/03/2021,72,1051862572052,100026728,COLCHONES PARAISO,Colchón Paraiso Bretania Firme Doble 140x190x2...,3/08/2020,Call center,Cliente manifiesta que con su colchon presenta...,CARLOS ALBERTO RUA GITIERREZ,Correo,Juan Pablo Morales Galindo,NaN,1/02/2021,False,NaN,NaN,NaN,CAROVASQ
2,587618,Pendiente por documentación,Pendiente Respuesta del Cliente,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Rosario Aidee Baquero Cristancho,CC-51749512,rosbac811@hotmail.com,7143103.0,3.163759e+09,28/01/2021,12/03/2021,124,1035782884933,100123642,COLCHONES PARAISO,Colchón Paraiso Bretania Doble STD 140x190x26 ...,28/05/2020,Call center,User comunica que la compra realizada el 28/05...,DANIELA MUNOZ RODAS,Correo,Juan Pablo Morales Galindo,NaN,2/02/2021,False,NaN,NaN,NaN,CAROVASQ
3,578985,Pendiente por documentación,NaN,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Kelly Alexandra Medina Gutierrez,CC-53008993,alexamegu@hotmail.com,NaN,3.214112e+09,21/01/2021,5/03/2021,292,1047970793278,100091311,COLCHONES PARAISO,Colchón Paraiso Bretania Sencillo 100*190 cm STD,18/07/2020,Call center,Se comunica la cliente para generar la garanti...,SANDRA LORENA ROJAS GARZON,Numero de contacto,César Augusto Álvarez Franco,NaN,22/01/2021,False,NaN,NaN,NaN,CAROVASQ
4,578523,Pendiente por documentación,NaN,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Cielo Piedad Agudelo Rendon,CC-39437386,cielo.agudelorendon@gmail.com,5705817.0,3.187077e+09,20/01/2021,4/03/2021,308,1017220276079,100156662,COLCHONES PARAISO,Colchón Paraiso Bretania Doble Cara Semidoble ...,12/03/2020,Call center,Cliente se comunica solicitando se genere tram...,Maria Fernanda Meneses Henao,Correo,César Augusto Álvarez Franco,NaN,27/01/2021,False,NaN,NaN,NaN,CAROVASQ


In [38]:
df.columns

Index(['Caso', 'Estado', 'Estado Gestión', 'Propietario del caso',
       'Marca de Ocurrencia', 'Categoria', 'Vendedor', 'Nombre de la cuenta',
       'Documento', 'Email de Contacto', 'Teléfono de Contacto',
       'Celular de Contacto', 'Fecha de apertura', 'Fecha esperada de cierre',
       'Antigüedad', 'No. de Pedido/Tirilla', 'Producto', 'Marca PLU',
       'Descripcion PLU', 'Fecha de Compra', 'Origen del caso', 'Descripción',
       'Creado por', 'Medio de respuesta',
       'Última modificación de caso realizada por', 'Medio de devolucion',
       'Fecha de la última modificación de caso', 'Solución Express',
       'Fecha de cierre', 'Detalle solución', 'Descripcion Solucion',
       'Alias del propietario del caso'],
      dtype='object')

In [39]:
col = ['Caso', 'Descripcion PLU', 'Descripción']
df = df[col]

In [40]:
import re
# Limpieza, carecteres especiales y signos de puntuacion
def limpieza_basica(body):
    body = body.lower() #todo en minuscula
    #body = re.sub(" \d+",' ', body) #Eliminar numeros
    #body = re.sub('[\d+]',' ', body) #quitar numeros dentro de palabras
    body = re.sub('\[#*.>=\]','',body) # Eliminar caracteres 
    #body = re.sub(r'[^\w\s]','',body) #eliminar signos de puntuacion
    #body = re.sub('[%s]' %re.escape(string.punctuation),' ', body) # Eliminar signos puntuacion
    body= re.sub(r"[\r\n]+",' ', body) # saltos de linea iguales
    body=re.sub("\s\s+" , " ", body) #quita varios espacios
    return body
limpiar = lambda x: limpieza_basica(x)

In [41]:
df["Des"] = df["Descripción"].apply(limpiar)

In [42]:
df.head(5)

,Caso,Descripcion PLU,Descripción,Des
0,479725,Colchon Paraíso Bora Doble 140x190x29 Una cara...,0000030a5170a9ff/TAMEJIHO: Cliente se comunica...,0000030a5170a9ff/tamejiho: cliente se comunica...
1,590248,Colchón Paraiso Bretania Firme Doble 140x190x2...,Cliente manifiesta que con su colchon presenta...,cliente manifiesta que con su colchon presenta...
2,587618,Colchón Paraiso Bretania Doble STD 140x190x26 ...,User comunica que la compra realizada el 28/05...,user comunica que la compra realizada el 28/05...
3,578985,Colchón Paraiso Bretania Sencillo 100*190 cm STD,Se comunica la cliente para generar la garanti...,se comunica la cliente para generar la garanti...
4,578523,Colchón Paraiso Bretania Doble Cara Semidoble ...,Cliente se comunica solicitando se genere tram...,cliente se comunica solicitando se genere tram...


In [43]:
# def limpieza(x): 
#     x=str(x).lower()
#     x=re.sub(r"([^a-zA-Z\s]+)",'',str(x)) # eliminar todos los caracteres que no son letras y cambiarlos por espacio en blanco
#     x=re.sub(r'xx+|1s+', '', str(x))
#     x=re.sub(r'xx+|1s+', '', str(x))
#     x=re.sub(r"jhgj|mmmmm|nj",'',str(x))
#     x=re.sub(r"cc",'',str(x))
#    #re.sub('\s+',' ',str(x)) 
#     return x
# df["Des"] = df["Des"].apply(lambda s: limpieza(s))

In [44]:
df["Des"] = df["Des"].str.strip()

In [45]:
#Quitarle los saltos de linea
df["Des"] = df["Des"].apply(lambda x:re.sub(r"\n",'',x))

In [46]:
df.Des

0       0000030a5170a9ff/tamejiho: cliente se comunica...
1       cliente manifiesta que con su colchon presenta...
2       user comunica que la compra realizada el 28/05...
3       se comunica la cliente para generar la garanti...
4       cliente se comunica solicitando se genere tram...
                              ...                        
2479    no prende . no funciona . por favor agilizar l...
2480    cliente nos informa que da imágenes distorsion...
2481    la cliente manifiesta que no carga con su prop...
2482    el cliente manifiesta que desde que le llego e...
2483    cliente nos indica que le aparecieron las line...
Name: Des, Length: 2484, dtype: object

In [47]:
df.isna().sum()

Caso                0
Descripcion PLU    48
Descripción         0
Des                 0
dtype: int64

In [48]:
Tipologia = pd.read_excel('TipoProductoGarantia.xlsx')
Tipologia.head()

,Caso,Producto,Problemas,Chisme
0,336699,headphone de iphone,No cumple funcion,NaN
1,343610,aspiradora robot,No cumple funcion,NaN
2,505804,freidora hyundai aire digital,Dificultad Encendido/ No enciende,solicita una solución rápida
3,366093,Maquina Peluquera Eléctrica Cortador de Cabello,Fallas en el funcionamiento,Señor quedo trasquilado
4,563000,combo cama,Novedad sonido: No tiene sonido,"compró colchón fantasía y le llegó paraíso, se..."


In [49]:
Tipologia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Caso       457 non-null    int64 
 1   Producto   453 non-null    object
 2   Problemas  449 non-null    object
 3   Chisme     352 non-null    object
dtypes: int64(1), object(3)
memory usage: 14.4+ KB


In [50]:
df.columns

Index(['Caso', 'Descripcion PLU', 'Descripción', 'Des'], dtype='object')

In [51]:
#Union de las datas
Datos = Tipologia.merge (df, on='Caso',how='left' )

In [52]:
Datos.sample(10)

,Caso,Producto,Problemas,Chisme,Descripcion PLU,Descripción,Des
511,451764,FREIDORA DE AIRE BIOCERAMIC OSTER CKSTAF7601,N|A,NaN,FREIDORA DE AIRE BIOCERAMIC OSTER CKSTAF7601,"Viene del caso 00306976, donde al cliente ya s...","viene del caso 00306976, donde al cliente ya s..."
334,399777,Celular Xiaomi Redmi Note 8 Pro 64Gb Azul,Problemas con pantalla,"Cliente nota un punto blanco en su producto, c...",Celular Xiaomi Redmi Note 8 Pro 64Gb Azul,Cliente se comunica para indicar que el produc...,cliente se comunica para indicar que el produc...
522,587618,Colchón Paraiso Bretania Doble STD 140x190x26 ...,Novedad de calidad,"inconveniente con la tela, se moteó",Colchón Paraiso Bretania Doble STD 140x190x26 ...,User comunica que la compra realizada el 28/05...,user comunica que la compra realizada el 28/05...
359,405147,Televisor Samsung Crystal 55 pulgadas UHD 4K S...,Problemas con pantalla,Presenta rayas,Televisor Samsung Crystal 55 pulgadas UHD 4K S...,Durante esta semana he notado que el Tv Samsun...,durante esta semana he notado que el tv samsun...
491,449235,"TV SMART 58"" 4K ULTRA HD SLIM PHILIPS 58 Pulga...",Problemas con pantalla,Presenta rayas,"TV SMART 58"" 4K ULTRA HD SLIM PHILIPS 58 Pulga...",Cliente se comunica solicitando garantía del p...,cliente se comunica solicitando garantía del p...
643,573367,Maquina de Coser Portatil Mini Electrica Sewin...,Fallas en el funcionamiento,perilla de tensión no funciona,Maquina de Coser Portatil Mini Electrica Sewin...,cliente se comunica por medio de chat solicita...,cliente se comunica por medio de chat solicita...
668,457050,Televisor LED Caixun 43 Pulgadas 109 Cms UHD S...,N|A,Producto en el almacén,Televisor LED Caixun 43 Pulgadas 109 Cms UHD S...,Se comunica la señora Yaneth de almacenes éxit...,se comunica la señora yaneth de almacenes éxit...
776,470290,Televisor LED Samsung 50 Pulgadas UHD 4K Smart...,Nota credito,NaN,Televisor LED Samsung 50 Pulgadas UHD 4K Smart...,la cliente estaba tramitando un caso por garan...,la cliente estaba tramitando un caso por garan...
415,461364,Colchon Doble Vitalic 140x190 cm,Novedad de calidad,Se esta hundiendo,Colchon Doble Vitalic 140x190 cm,Cliente se comunica por que hace 20 dias el Co...,cliente se comunica por que hace 20 dias el co...
19,562940,producto madera,Novedad de calidad,al parecer tenía otro caso cerrado por el mism...,ESCRITORIO ARCHIVADOR INVAL ES4603,Cliente se comunica indicando que tenia caso a...,cliente se comunica indicando que tenia caso a...


In [53]:
Datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 0 to 817
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Caso             818 non-null    int64 
 1   Producto         811 non-null    object
 2   Problemas        810 non-null    object
 3   Chisme           645 non-null    object
 4   Descripcion PLU  804 non-null    object
 5   Descripción      812 non-null    object
 6   Des              812 non-null    object
dtypes: int64(1), object(6)
memory usage: 51.1+ KB


In [54]:
Datos.Des.unique()

array(['la cliente solicita garantía de headphone de iphone no funciona el microfono',
       'se retipifica caso. la cliente indica que el producto se queda girando en su propio eje y no hace más nada. id: 00000302907386e9',
       '0000030b859bedb0/tamejiho: cliente se comunica reportando que el producto freidora aire digital 2.6lts plu: 1666635 no esta funcionando adecuadamente pues ayer fue a prenderlo y no le prendió, dice que el producto venia funcionando normal, pide se le revise y se le repare, o se le cambie el producto lo que aplique para el caso, de la manera mas atenta se les pide contactar y brindar solución en la menor brevedad posible.muchas gracias.se le piden evidencias de la falla.',
       '0000030b859bedb0/tamejiho: cliente se comunica reportando que el producto freidora aire digital 2.6lts plu: 1666635 no esta funcionando adecuadamente pues ayer fue a prenderlo y no le prendió, dice que el producto venia funcionando normal, pide se le revise y se le repare, o se le

In [55]:
#Problemas con producto o servicio
colp = ['Caso','Problemas', 'Des']
df = Datos[colp]
df.head(50)

,Caso,Problemas,Des
0,336699,No cumple funcion,la cliente solicita garantía de headphone de i...
1,336699,No cumple funcion,la cliente solicita garantía de headphone de i...
2,343610,No cumple funcion,se retipifica caso. la cliente indica que el p...
3,343610,No cumple funcion,se retipifica caso. la cliente indica que el p...
4,505804,Dificultad Encendido/ No enciende,0000030b859bedb0/tamejiho: cliente se comunica...
5,505804,Dificultad Encendido/ No enciende,0000030b859bedb0/tamejiho: cliente se comunica...
6,505804,Dificultad Encendido/ No enciende,0000030b859bedb0/tamejiho: cliente se comunica...
7,505804,Dificultad Encendido/ No enciende,0000030b859bedb0/tamejiho: cliente se comunica...
8,366093,Fallas en el funcionamiento,cliente se comunica para solicitar garantía de...
9,366093,Fallas en el funcionamiento,cliente se comunica para solicitar garantía de...


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 0 to 817
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Caso       818 non-null    int64 
 1   Problemas  810 non-null    object
 2   Des        812 non-null    object
dtypes: int64(1), object(2)
memory usage: 25.6+ KB


In [57]:
df = df.drop_duplicates( "Caso" , keep='first')

In [58]:
#Valores nulos en la categoria
df.isna().sum()

Caso         0
Problemas    8
Des          6
dtype: int64

In [59]:
df = df.dropna()

In [60]:
df.isna().sum()

Caso         0
Problemas    0
Des          0
dtype: int64

In [61]:
df.columns


Index(['Caso', 'Problemas', 'Des'], dtype='object')

In [62]:
df.Problemas.unique()

array(['No cumple funcion ', 'Dificultad Encendido/ No enciende',
       'Fallas en el funcionamiento', 'Novedad sonido: No tiene sonido',
       'Novedad de calidad', 'N|A', 'Nota credito',
       'Tramite Fallido con Marca', 'Problemas con pantalla', 'Averia',
       'Novedad en conexiones'], dtype=object)

In [63]:
possible_labels = df.Problemas.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Averia': 9,
 'Dificultad Encendido/ No enciende': 1,
 'Fallas en el funcionamiento': 2,
 'No cumple funcion ': 0,
 'Nota credito': 6,
 'Novedad de calidad': 4,
 'Novedad en conexiones': 10,
 'Novedad sonido: No tiene sonido': 3,
 'N|A': 5,
 'Problemas con pantalla': 8,
 'Tramite Fallido con Marca': 7}

In [64]:
df['label'] = df.Problemas.replace(label_dict)

In [65]:
df.columns

Index(['Caso', 'Problemas', 'Des', 'label'], dtype='object')

In [66]:
df['Problemas'].value_counts()

Fallas en el funcionamiento          117
N|A                                   64
Problemas con pantalla                55
Dificultad Encendido/ No enciende     55
Novedad de calidad                    44
No cumple funcion                     27
Novedad sonido: No tiene sonido       17
Nota credito                          14
Tramite Fallido con Marca             13
Averia                                11
Novedad en conexiones                 10
Name: Problemas, dtype: int64

In [67]:
co = ['Des', 'label', 'Problemas',]
df = df[co]
df.head()

,Des,label,Problemas
0,la cliente solicita garantía de headphone de i...,0,No cumple funcion
2,se retipifica caso. la cliente indica que el p...,0,No cumple funcion
4,0000030b859bedb0/tamejiho: cliente se comunica...,1,Dificultad Encendido/ No enciende
8,cliente se comunica para solicitar garantía de...,2,Fallas en el funcionamiento
10,el cliente se comunica informando que el produ...,3,Novedad sonido: No tiene sonido


In [68]:
class_counts = df['label'].value_counts()
class_counts

2     117
5      64
8      55
1      55
4      44
0      27
3      17
6      14
7      13
9      11
10     10
Name: label, dtype: int64

In [69]:
class_weights = len(class_counts)/class_counts
df = df.sample(
    n=class_counts.max()*len(class_counts),
    weights=df['label'].map(class_weights), 
    replace=True)

In [70]:
df['label'].value_counts()

1     139
4     126
3     126
6     124
10    122
8     119
5     114
7     108
9     106
2     106
0      97
Name: label, dtype: int64

In [71]:
df['Problemas'].value_counts()

Dificultad Encendido/ No enciende    139
Novedad de calidad                   126
Novedad sonido: No tiene sonido      126
Nota credito                         124
Novedad en conexiones                122
Problemas con pantalla               119
N|A                                  114
Tramite Fallido con Marca            108
Averia                               106
Fallas en el funcionamiento          106
No cumple funcion                     97
Name: Problemas, dtype: int64

## Setup BERT
dccuchile/bert-base-spanish-wwm-cased

In [72]:
#SOLUCION
import importlib, pkg_resources, tokenizers
importlib.reload(pkg_resources)
importlib.reload(tokenizers) 

<module 'tokenizers' from '/usr/local/lib/python3.6/dist-packages/tokenizers/__init__.py'>

In [73]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

In [74]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [75]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(
  df,
  test_size=0.3,
  random_state= 42
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state= 42
)

df_train.shape, df_val.shape, df_test.shape

((900, 3), (193, 3), (194, 3))

In [76]:
# Optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 3
model_args.learning_rate = 1e-4

In [77]:
# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=11,
    args=model_args
) 

INFO:filelock:Lock 140268444950312 acquired on /root/.cache/huggingface/transformers/cb7cedb04246e225d56ba26d207f1d1809b31a9bbe9b63103371d835c6ac0502.4b9caa3a5bac782a13e32cb17f725209cfd0e6e99cdc8ff68d62d43fe1838167.lock


Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

INFO:filelock:Lock 140268444950312 released on /root/.cache/huggingface/transformers/cb7cedb04246e225d56ba26d207f1d1809b31a9bbe9b63103371d835c6ac0502.4b9caa3a5bac782a13e32cb17f725209cfd0e6e99cdc8ff68d62d43fe1838167.lock
INFO:filelock:Lock 140268444950368 acquired on /root/.cache/huggingface/transformers/52382cbe7c1587c6b588daa81eaf247c5e2ad073d42b52192a8cd4202e7429b6.f6102c422ce4707eef3f64ba8a47be5cb5abf284c414bdc20c7c22f2e5336704.lock


Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

INFO:filelock:Lock 140268444950368 released on /root/.cache/huggingface/transformers/52382cbe7c1587c6b588daa81eaf247c5e2ad073d42b52192a8cd4202e7429b6.f6102c422ce4707eef3f64ba8a47be5cb5abf284c414bdc20c7c22f2e5336704.lock
Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

...

INFO:filelock:Lock 140268304182352 released on /root/.cache/huggingface/transformers/8411af1ad9b6e0857f0ee6b7a98b3fd432de9e8f571eee2b4f63269cece16ac4.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
INFO:filelock:Lock 140268304182352 acquired on /root/.cache/huggingface/transformers/9848a00af462c42dfb4ec88ef438fbab5256330f7f6f50badc48d277f9367d49.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO:filelock:Lock 140268304182352 released on /root/.cache/huggingface/transformers/9848a00af462c42dfb4ec88ef438fbab5256330f7f6f50badc48d277f9367d49.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
INFO:filelock:Lock 140271180655248 acquired on /root/.cache/huggingface/transformers/ca34e6c1251888a8ed98da2a454f869d28e3438eef67c2f93aa8133459ac08a3.a6c00251b9344c189e2419373d6033016d0cd3d87ea59f6c86069046ac81956d.lock


Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

INFO:filelock:Lock 140271180655248 released on /root/.cache/huggingface/transformers/ca34e6c1251888a8ed98da2a454f869d28e3438eef67c2f93aa8133459ac08a3.a6c00251b9344c189e2419373d6033016d0cd3d87ea59f6c86069046ac81956d.lock


In [78]:
# Train the model
model.train_model(df_train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_11_900


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/113 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 3:   0%|          | 0/113 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/113 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


(339, 0.6504351263261431)

In [79]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(df_test, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_194


Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9107263487626972, 'f1': 0.9175257731958762, 'acc': 0.9175257731958762, 'eval_loss': 0.3204555333219469}


{'acc': 0.9175257731958762,
 'eval_loss': 0.3204555333219469,
 'f1': 0.9175257731958762,
 'mcc': 0.9107263487626972}

In [80]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(df_val)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_193


Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9103704781498526, 'f1': 0.9175257731958762, 'acc': 0.9175257731958762, 'eval_loss': 0.3450230381824076}


In [81]:
pred = model.predict(df_test['Des'].values)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_194


  0%|          | 0/25 [00:00<?, ?it/s]

In [82]:
pred = list(pred[0])


In [83]:
y = df_test['label']
y =list(y)

In [84]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, pred)

array([[13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  3, 11,  0,  3,  0,  0,  0,  2,  0,  0],
       [ 0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0, 23,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  2,  0,  0, 17,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 19,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 15,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 14,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16]])

In [85]:
from sklearn.metrics import classification_report
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90        13
           1       0.81      1.00      0.89        17
           2       0.85      0.50      0.63        22
           3       1.00      1.00      1.00        20
           4       0.88      0.96      0.92        24
           5       1.00      0.85      0.92        20
           6       1.00      1.00      1.00        19
           7       1.00      1.00      1.00        13
           8       0.83      0.94      0.88        16
           9       0.93      1.00      0.97        14
          10       1.00      1.00      1.00        16

    accuracy                           0.92       194
   macro avg       0.92      0.93      0.92       194
weighted avg       0.92      0.92      0.91       194



In [86]:
testo = df_test.label

In [87]:
class_list = ['No cumple funcion ', 'Dificultad Encendido/ No enciende',
       'Fallas en el funcionamiento', 'Novedad sonido: No tiene sonido',
       'Novedad de calidad', 'N|A', 'Nota credito',
       'Tramite Fallido con Marca', 'Problemas con pantalla', 'Averia',
       'Novedad en conexiones']
predictions, raw_outputs = model.predict(["el producto me llego sin ruedas, y el tablo de la silla está hundido"])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Novedad de calidad


In [88]:
predictions, raw_outputs = model.predict(["El cliente se  comunica indicando que un audífono ya no esta funcionando el cual desea que le brinden la garantía del producto.Se le indica que pasa por validación 100353973 Audifonos Manos Libres Apple Earpods Conector Lightning"])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Novedad sonido: No tiene sonido


In [89]:
predictions, raw_outputs = model.predict(["dice que la pantalla de su computador todo en uno no prende o titila de varios colores y no da imagen y de un momento a otro se arregla.dice que necesita garantía de su producto dado que la tiene por un año completo.. "])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Problemas con pantalla


In [90]:
predictions, raw_outputs = model.predict(["cliente se comunica por que hizo un pedido de un secador el dia 24 de septiembre  y el dia 4 de octubre la segunda vez que lo utilizo le dejo de funcionar le hizo corto cuando lo conecto"])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Fallas en el funcionamiento


In [91]:
predictions, raw_outputs = model.predict(["usuario se comunica con nosotros indicando que necesita que se le realice la devolución de su dinero ya que una de sus  maletas se encuentran descocidas.quiere la devolución."])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Novedad de calidad


In [92]:
predictions, raw_outputs = model.predict(["cliente se comunica indicando que el día 29 de agosto realizó la compra de un Combo Colchon Semidoble Blue Balanced 120x190 cm+ Basecama+ Protector+ Almohada, manifiesta que hace poco está presentando un inconveniente con el producto, informa que el colchon se está destiñendo  y tiene unas manchas amarillas, solicita por favor un trámite de garantía"])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Novedad de calidad


In [93]:
predictions, raw_outputs = model.predict(["Cliente reporta la falla de producto Televisor LED Caixun 43 Pulgadas 109 Cms UHD Smart CX43P28USM, al momento de encenderlo le salen unas rayas en la pantalla y despues queda gris. Solicita el tramite de la garantia"])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Problemas con pantalla


In [94]:
predictions, raw_outputs = model.predict(["Cliente se comunica para indica que recibió el producto Elíptica 7 en 1 Home Sale Gris Negro con plu: 100019910 el día 08-11-2020 ella contrato a una persona profesional para que esta persona la instalara, al momento de hacerlo se dan cuenta que la pieza central, el tubo que soporta el panel del display, el tubo central, esta torcida, y la persona que lo arma indica que es un daño de fabrica, por lo tanto la cliente accede al tramite de garantía del producto ya que no aplica para avería por el tiempo en que lo reporto, SE LE INDICA QUE ENVIE EVIDENCIAS DEL INCONVENIENTE PARA QUE SE VERIFIQUE Y SE VALIDE Y LE BRINDEN UNA SOLUCION. "])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Novedad de calidad


In [95]:
predictions, raw_outputs = model.predict(["Se comunica cliente pidiendo la garantía del celular xiaomi ella indica que de un momento a otro el celular dejó de prender."])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Dificultad Encendido/ No enciende


<center><img src="https://www.philschmid.de/static/6a626886832ad68c0ee467e7fc66fa4c/b1001/meme.png"></center>

In [96]:
import torch

In [97]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [98]:
save_model('outputs', 'betochampions')

In [99]:
!tar -zxvf ./betochampions.tar.gz

outputs/config.json
outputs/special_tokens_map.json
outputs/pytorch_model.bin
outputs/model_args.json
outputs/tokenizer_config.json
outputs/training_args.bin
outputs/eval_results.txt
outputs/vocab.txt


In [100]:
!rm -rf outputs

In [101]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('betochampions')

In [102]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "outputs/",
    num_labels=11,
    args=train_args
)

In [103]:
class_names = ['No cumple funcion ', 'Dificultad Encendido/ No enciende',
       'Fallas en el funcionamiento', 'Novedad sonido: No tiene sonido',
       'Novedad de calidad', 'N|A', 'Nota credito',
       'Tramite Fallido con Marca', 'Problemas con pantalla', 'Averia',
       'Novedad en conexiones']

In [104]:
predictions, raw_outputs = model.predict(["Cliente indica que desde el monto que lo recibió en Noviembre uno de los audífonos no funcionaba de forma correcta, tenía que hacerle presión para que encendiera bien y el miércoles 09 de Diciembre dejó de funcionar totalmente."])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Fallas en el funcionamiento


In [106]:
predictions, raw_outputs = model.predict(["Cliente se comunica indicando que compró una patineta eléctrica el mes de Abril y que el día de ayer 24 de Noviembre la patineta ya no enciende, quiere que se realice un tramite por garantía."])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Dificultad Encendido/ No enciende


In [107]:
predictions, raw_outputs = model.predict(["Cliente manifiesta que el COLCHON DOBLE BLUE LEXUS 140X190 CM FANTASIA CCBL140190 PLU: 442172 cliente  indica que por los lados se le salen las barillas y que el producto se encuentra en quibdo en la direccion Barrio medrano sector las terrazas. "])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Novedad de calidad


In [108]:
predictions, raw_outputs = model.predict(["Cliente manifiesta que el celular se le dañaron los botones"])
print(class_list[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_1


  0%|          | 0/1 [00:00<?, ?it/s]

Fallas en el funcionamiento


In [109]:
ClassificationModel

simpletransformers.classification.classification_model.ClassificationModel

In [110]:
Predicciones = pd.read_excel("TramiteGarantia.xlsx")
Predicciones.head()

,Caso,Estado,Estado Gestión,Propietario del caso,Marca de Ocurrencia,Categoria,Vendedor,Nombre de la cuenta,Documento,Email de Contacto,Teléfono de Contacto,Celular de Contacto,Fecha de apertura,Fecha esperada de cierre,Antigüedad,No. de Pedido/Tirilla,Producto,Marca PLU,Descripcion PLU,Fecha de Compra,Origen del caso,Descripción,Creado por,Medio de respuesta,Última modificación de caso realizada por,Medio de devolucion,Fecha de la última modificación de caso,Solución Express,Fecha de cierre,Detalle solución,Descripcion Solucion,Alias del propietario del caso
0,479725,Pendiente por documentación,Reasignado N2,Lina Marcela Bohorquez Gonzalez,EXITO.COM,Marketplace,Colchones Paraiso 1,Kevis Manuel Pitalua Castro,CC-1070813521,elvivekt@hotmail.com,NaN,3.117452e+09,18/11/2020,7/01/2021,1829,1041533392806,100126160,COLCHONES PARAISO,Colchon Paraíso Bora Doble 140x190x29 Una cara...,21/06/2020,Call center,0000030a5170a9ff/TAMEJIHO: Cliente se comunica...,TATIANA ANDREA MEJÍA HOYOS,Correo,Cristian Omar Montiel Cermeño,NaN,2/02/2021,False,NaN,NaN,NaN,linaboho
1,590248,Pendiente por documentación,Pendiente Respuesta del Cliente,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Bejarano Sandra Patric Beltran,CC-40447782,jsmarin04@hotmail.com,NaN,3.118258e+09,30/01/2021,15/03/2021,72,1051862572052,100026728,COLCHONES PARAISO,Colchón Paraiso Bretania Firme Doble 140x190x2...,3/08/2020,Call center,Cliente manifiesta que con su colchon presenta...,CARLOS ALBERTO RUA GITIERREZ,Correo,Juan Pablo Morales Galindo,NaN,1/02/2021,False,NaN,NaN,NaN,CAROVASQ
2,587618,Pendiente por documentación,Pendiente Respuesta del Cliente,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Rosario Aidee Baquero Cristancho,CC-51749512,rosbac811@hotmail.com,7143103.0,3.163759e+09,28/01/2021,12/03/2021,124,1035782884933,100123642,COLCHONES PARAISO,Colchón Paraiso Bretania Doble STD 140x190x26 ...,28/05/2020,Call center,User comunica que la compra realizada el 28/05...,DANIELA MUNOZ RODAS,Correo,Juan Pablo Morales Galindo,NaN,2/02/2021,False,NaN,NaN,NaN,CAROVASQ
3,578985,Pendiente por documentación,NaN,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Kelly Alexandra Medina Gutierrez,CC-53008993,alexamegu@hotmail.com,NaN,3.214112e+09,21/01/2021,5/03/2021,292,1047970793278,100091311,COLCHONES PARAISO,Colchón Paraiso Bretania Sencillo 100*190 cm STD,18/07/2020,Call center,Se comunica la cliente para generar la garanti...,SANDRA LORENA ROJAS GARZON,Numero de contacto,César Augusto Álvarez Franco,NaN,22/01/2021,False,NaN,NaN,NaN,CAROVASQ
4,578523,Pendiente por documentación,NaN,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Cielo Piedad Agudelo Rendon,CC-39437386,cielo.agudelorendon@gmail.com,5705817.0,3.187077e+09,20/01/2021,4/03/2021,308,1017220276079,100156662,COLCHONES PARAISO,Colchón Paraiso Bretania Doble Cara Semidoble ...,12/03/2020,Call center,Cliente se comunica solicitando se genere tram...,Maria Fernanda Meneses Henao,Correo,César Augusto Álvarez Franco,NaN,27/01/2021,False,NaN,NaN,NaN,CAROVASQ


In [111]:
columnas = ['Caso', 'Descripción']
self_df = Predicciones[columnas]
self_df.head()

,Caso,Descripción
0,479725,0000030a5170a9ff/TAMEJIHO: Cliente se comunica...
1,590248,Cliente manifiesta que con su colchon presenta...
2,587618,User comunica que la compra realizada el 28/05...
3,578985,Se comunica la cliente para generar la garanti...
4,578523,Cliente se comunica solicitando se genere tram...


In [112]:
import re
# Limpieza, carecteres especiales y signos de puntuacion
def limpieza_basica(body):
    body = body.lower() #todo en minuscula
    #body = re.sub(" \d+",' ', body) #Eliminar numeros
    #body = re.sub('[\d+]',' ', body) #quitar numeros dentro de palabras
    body = re.sub('\[#*.>=\]','',body) # Eliminar caracteres 
    #body = re.sub(r'[^\w\s]','',body) #eliminar signos de puntuacion
    #body = re.sub('[%s]' %re.escape(string.punctuation),' ', body) # Eliminar signos puntuacion
    body= re.sub(r"[\r\n]+",' ', body) # saltos de linea iguales
    body=re.sub("\s\s+" , " ", body) #quita varios espacios
    return body
limpiar = lambda x: limpieza_basica(x)

In [113]:
self_df["Des"] = self_df["Descripción"].apply(limpiar)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [114]:
self_df.head()

,Caso,Descripción,Des
0,479725,0000030a5170a9ff/TAMEJIHO: Cliente se comunica...,0000030a5170a9ff/tamejiho: cliente se comunica...
1,590248,Cliente manifiesta que con su colchon presenta...,cliente manifiesta que con su colchon presenta...
2,587618,User comunica que la compra realizada el 28/05...,user comunica que la compra realizada el 28/05...
3,578985,Se comunica la cliente para generar la garanti...,se comunica la cliente para generar la garanti...
4,578523,Cliente se comunica solicitando se genere tram...,cliente se comunica solicitando se genere tram...


In [115]:
self_df.Des.isna().sum()

0

In [117]:
self_df.columns

Index(['Caso', 'Descripción', 'Des'], dtype='object')

In [119]:
cols = ['Caso', 'Des']
self_df = self_df[cols]
self_df.head()

,Caso,Des
0,479725,0000030a5170a9ff/tamejiho: cliente se comunica...
1,590248,cliente manifiesta que con su colchon presenta...
2,587618,user comunica que la compra realizada el 28/05...
3,578985,se comunica la cliente para generar la garanti...
4,578523,cliente se comunica solicitando se genere tram...


In [120]:
class_names = ['No cumple funcion ', 'Dificultad Encendido/ No enciende',
       'Fallas en el funcionamiento', 'Novedad sonido: No tiene sonido',
       'Novedad de calidad', 'N|A', 'Nota credito',
       'Tramite Fallido con Marca', 'Problemas con pantalla', 'Averia',
       'Novedad en conexiones']

In [121]:
predictions, raw_outputs = model.predict(self_df['Des'])
print(class_names[predictions[0]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_11_2484


  0%|          | 0/311 [00:00<?, ?it/s]

Novedad de calidad


In [122]:
self_df['Pred'] = predictions

In [123]:
self_df

,Caso,Des,Pred
0,479725,0000030a5170a9ff/tamejiho: cliente se comunica...,4
1,590248,cliente manifiesta que con su colchon presenta...,4
2,587618,user comunica que la compra realizada el 28/05...,4
3,578985,se comunica la cliente para generar la garanti...,4
4,578523,cliente se comunica solicitando se genere tram...,4
...,...,...,...
2479,532775,no prende . no funciona . por favor agilizar l...,1
2480,532926,cliente nos informa que da imágenes distorsion...,2
2481,533030,la cliente manifiesta que no carga con su prop...,5
2482,573140,el cliente manifiesta que desde que le llego e...,9


In [124]:
Garantia = pd.merge(Predicciones ,self_df,on='Caso',how='left')
Garantia.head()

,Caso,Estado,Estado Gestión,Propietario del caso,Marca de Ocurrencia,Categoria,Vendedor,Nombre de la cuenta,Documento,Email de Contacto,Teléfono de Contacto,Celular de Contacto,Fecha de apertura,Fecha esperada de cierre,Antigüedad,No. de Pedido/Tirilla,Producto,Marca PLU,Descripcion PLU,Fecha de Compra,Origen del caso,Descripción,Creado por,Medio de respuesta,Última modificación de caso realizada por,Medio de devolucion,Fecha de la última modificación de caso,Solución Express,Fecha de cierre,Detalle solución,Descripcion Solucion,Alias del propietario del caso,Des,Pred
0,479725,Pendiente por documentación,Reasignado N2,Lina Marcela Bohorquez Gonzalez,EXITO.COM,Marketplace,Colchones Paraiso 1,Kevis Manuel Pitalua Castro,CC-1070813521,elvivekt@hotmail.com,NaN,3.117452e+09,18/11/2020,7/01/2021,1829,1041533392806,100126160,COLCHONES PARAISO,Colchon Paraíso Bora Doble 140x190x29 Una cara...,21/06/2020,Call center,0000030a5170a9ff/TAMEJIHO: Cliente se comunica...,TATIANA ANDREA MEJÍA HOYOS,Correo,Cristian Omar Montiel Cermeño,NaN,2/02/2021,False,NaN,NaN,NaN,linaboho,0000030a5170a9ff/tamejiho: cliente se comunica...,4
1,479725,Pendiente por documentación,Reasignado N2,Lina Marcela Bohorquez Gonzalez,EXITO.COM,Marketplace,Colchones Paraiso 1,Kevis Manuel Pitalua Castro,CC-1070813521,elvivekt@hotmail.com,NaN,3.117452e+09,18/11/2020,7/01/2021,1829,1041533392806,100126160,COLCHONES PARAISO,Colchon Paraíso Bora Doble 140x190x29 Una cara...,21/06/2020,Call center,0000030a5170a9ff/TAMEJIHO: Cliente se comunica...,TATIANA ANDREA MEJÍA HOYOS,Correo,Cristian Omar Montiel Cermeño,NaN,2/02/2021,False,NaN,NaN,NaN,linaboho,0000030a5170a9ff/tamejiho: cliente se comunica...,4
2,590248,Pendiente por documentación,Pendiente Respuesta del Cliente,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Bejarano Sandra Patric Beltran,CC-40447782,jsmarin04@hotmail.com,NaN,3.118258e+09,30/01/2021,15/03/2021,72,1051862572052,100026728,COLCHONES PARAISO,Colchón Paraiso Bretania Firme Doble 140x190x2...,3/08/2020,Call center,Cliente manifiesta que con su colchon presenta...,CARLOS ALBERTO RUA GITIERREZ,Correo,Juan Pablo Morales Galindo,NaN,1/02/2021,False,NaN,NaN,NaN,CAROVASQ,cliente manifiesta que con su colchon presenta...,4
3,587618,Pendiente por documentación,Pendiente Respuesta del Cliente,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Rosario Aidee Baquero Cristancho,CC-51749512,rosbac811@hotmail.com,7143103.0,3.163759e+09,28/01/2021,12/03/2021,124,1035782884933,100123642,COLCHONES PARAISO,Colchón Paraiso Bretania Doble STD 140x190x26 ...,28/05/2020,Call center,User comunica que la compra realizada el 28/05...,DANIELA MUNOZ RODAS,Correo,Juan Pablo Morales Galindo,NaN,2/02/2021,False,NaN,NaN,NaN,CAROVASQ,user comunica que la compra realizada el 28/05...,4
4,578985,Pendiente por documentación,NaN,CAROLINA VASQUEZ EUSE,EXITO.COM,Marketplace,Colchones Paraiso 1,Kelly Alexandra Medina Gutierrez,CC-53008993,alexamegu@hotmail.com,NaN,3.214112e+09,21/01/2021,5/03/2021,292,1047970793278,100091311,COLCHONES PARAISO,Colchón Paraiso Bretania Sencillo 100*190 cm STD,18/07/2020,Call center,Se comunica la cliente para generar la garanti...,SANDRA LORENA ROJAS GARZON,Numero de contacto,César Augusto Álvarez Franco,NaN,22/01/2021,False,NaN,NaN,NaN,CAROVASQ,se comunica la cliente para generar la garanti...,4


In [126]:
Garantia.loc[(Garantia['Pred'] ==9, 'Pred')] = 'Averia'
Garantia.loc[(Garantia['Pred'] ==1, 'Pred')] = ' Dificultad Encendido/ No enciende'
Garantia.loc[(Garantia['Pred'] ==2, 'Pred')] = ' Fallas en el funcionamiento'
Garantia.loc[(Garantia['Pred'] ==0, 'Pred')] = ' No cumple funcion '
Garantia.loc[(Garantia['Pred'] ==6, 'Pred')] = ' Nota credito'
Garantia.loc[(Garantia['Pred'] ==4, 'Pred')] = ' Novedad de calidad'
Garantia.loc[(Garantia['Pred'] ==10, 'Pred')] = ' Novedad en conexiones'
Garantia.loc[(Garantia['Pred'] ==3, 'Pred')] = ' Novedad sonido:  No tiene sonido'
Garantia.loc[(Garantia['Pred'] ==5, 'Pred')] = ' N|A'
Garantia.loc[(Garantia['Pred'] ==8, 'Pred')] = ' Problemas con pantalla'
Garantia.loc[(Garantia['Pred'] ==7, 'Pred')] = ' Tramite Fallido con Marca'


In [127]:
Garantia.to_excel('ProductoGarantia.xlsx')